# Unrestricted question based resolution of anaphoras   

In [1]:
import sys
sys.path.append("../")

In [2]:
import openai
import configparser

from llm_library.openai import configure_azure_model
from llm_library.openai import ask_openai

from pandas import read_csv
from pandas import DataFrame
from tqdm.auto import tqdm 
from math import isnan

## I. Set up Azure API 

In [3]:
config = configparser.ConfigParser()
status = config.read('../model_configurations/azure.ini') 
assert status == ['../model_configurations/azure.ini']

In [4]:
gpt_conf = configure_azure_model(config)
print(f"Going to use model {gpt_conf.model_name}")
print("Test")
print("Q: What is time?")
print(f"A: {ask_openai(gpt_conf, prompt='What is time?', stop='.')}")

Going to use model gpt-35-turbo
Test
Q: What is time?
A: Time is a fundamental concept used to measure the sequence and duration of events


## II. Direct anaphore resolution with ChatGPT  

In [5]:
tbl = read_csv('input_data/revealing_questions.csv')
tbl.head()

,index,label,pronoun,pronoun_lemma,pronoun_sentence,question,question_fix
0,0,1,neile,['see'],Inimeste kaasamine ettevõtte tuleviku planeeri...,kellele tagasiside andmine ja nende arendamine ?,kellele tagasiside andmine ?
1,1,1,nende,['tema'],Inimeste kaasamine ettevõtte tuleviku planeeri...,kelle arendamine ?,NaN
2,2,1,kes,"['kes', 'kes']","1 ) Eesti kodaniku abikaasat , **kes** taotl...",kes taotleb elamisluba käesoleva seaduse § 12 ...,NaN
3,3,1,millega,"['mis', 'mis']",Viimasel etapil hoidis tiirudes kõik märgid ma...,millega Eesti teenis 10 MK punkti ?,NaN
4,4,1,nad,['tema'],"Õnneks mehed siiski päästeti , sest **nad** ...",kes taipasid kiiresti telefonist SIM-kaardi ee...,NaN


###  Constructed question

In [6]:
prompt = (
    "'{sentence}'\n"
    'Answer the following question in JSON format.  Key value should be "answer" and entity value should be only one word.\n'
    "'{question}'\n"
    "The entity value should be exactly as it is written in the text.")

In [7]:
result = tbl[['pronoun_sentence', 'pronoun', 'question']].reset_index(names='index').assign(result=None)
for i,  (_, sentence, pronoun, question, _) in tqdm(result.iterrows(), total=len(result)):
    full_prompt = prompt.format(sentence=sentence.replace('**', ''), question=question)
    result.loc[i, 'result'] = ask_openai(gpt_conf, full_prompt)

  0%|          | 0/1071 [00:00<?, ?it/s]

In [8]:
result

,index,pronoun_sentence,pronoun,question,result
0,0,Inimeste kaasamine ettevõtte tuleviku planeeri...,neile,kellele tagasiside andmine ja nende arendamine ?,"{""answer"": ""inimestele""}"
1,1,Inimeste kaasamine ettevõtte tuleviku planeeri...,nende,kelle arendamine ?,None
2,2,"1 ) Eesti kodaniku abikaasat , **kes** taotl...",kes,kes taotleb elamisluba käesoleva seaduse § 12 ...,None
3,3,Viimasel etapil hoidis tiirudes kõik märgid ma...,millega,millega Eesti teenis 10 MK punkti ?,None
4,4,"Õnneks mehed siiski päästeti , sest **nad** ...",nad,kes taipasid kiiresti telefonist SIM-kaardi ee...,None
...,...,...,...,...,...
1066,1066,Tallinnale annetas selle Taani kuningas Valdem...,kes,kes kasutas annetuseks üht oma vapivälja ?,None
1067,1067,Kuid ärgem unustagem peegleid : need kolm läik...,need,mis kus teadupärast olemas on ?,None
1068,1068,Taustal oli ka riigis ülipopulaarne Poola päri...,kelle,kelle mõju ?,None
1069,1069,Sa ei viska aga pärast hoolikalt planeeritud m...,ta,"kes paned oma riidekappi , mis on esimene koht...",None


In [9]:
result.to_csv('output_data/anaphora_resolution_with_free_answer_and_constructed_question.csv')

### Manually corrected question

In [10]:
result = tbl[['pronoun_sentence', 'pronoun', 'question_fix']].reset_index(names='index').assign(result=None)
for i,  (_, sentence, pronoun, question, _) in tqdm(result.iterrows(), total=len(result)):
    if not isinstance(question, str):
        continue
    full_prompt = prompt.format(sentence=sentence.replace('**', ''), question=question)
    result.loc[i, 'result'] = ask_openai(gpt_conf, full_prompt)

  0%|          | 0/1071 [00:00<?, ?it/s]

In [11]:
result.to_csv('output_data/anaphora_resolution_with_free_answer_and_manually_corrected_questions.csv')